# Тестовое задание

Для начала выгрузим данные из excel и проверим их качество

In [1]:
import pandas as pd

data = pd.read_excel('data.xlsx',sheet_name = 'Сводный отчёт') # Импортируем данные

RR = pd.read_excel('data.xlsx',sheet_name = 'Разнарядки')

data.info() # Проверяем корректность данных

RR.info() # Скорее всего качество данных в порядке

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250818 entries, 0 to 250817
Data columns (total 33 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   Стадия                       250818 non-null  object        
 1   Статус                       250818 non-null  object        
 2   Акция                        250818 non-null  object        
 3   Начало                       250818 non-null  datetime64[ns]
 4   Окончание                    250818 non-null  datetime64[ns]
 5   Вид акции                    250818 non-null  object        
 6   Группа                       250818 non-null  object        
 7   Подгруппа                    250818 non-null  object        
 8   Код СВУ                      250818 non-null  int64         
 9   Филиал                       250818 non-null  object        
 10  Регион                       250818 non-null  object        
 11  Формат                    

Теперь приступим к ответам на вопросы.

# Первый вопрос:
Как вы считаете, достаточный ли набор параметров используется для принятия решения о продлении?

В файле было указано три параметра, по которым принимается решение. К этим параметрам можно добавить динамику эффективности промо. Имеется ввиду то, что если время проведения промо будет три недели, и только к концу третьей недели эффективность промо возрастёт, то имеет смысл продлить её, так как имеет место вероятность, что пик привлечения клиентов и продаж будет ещё впереди. 

Также стоит учитывать тот факт, что промо могут быть связаны с определёнными событиями, такими как новый год или какой-то другой праздник, и продлевать их, в том виде в каком они есть сейчас есть, не всегда хорошо. Зачастую лучше будет как-то переделать их или изменить.

Данных пяти параметров в большинстве случаев будет достаточно.

# Второй вопрос: 
Какой остаток на дату окончания промо корректнее рассматривать: фактический запас на магазине или разницу между разнорядкой и фактом продаж магазина? 

Здесь по сути стоит вопрос о том, учитывать ли только изменение количества товара в силу продаж или же рассматривать все факторы изменения количества товара (например, списание из-за срока годности). Для оценки эффективности промо будет корректнее использовать разницу между разнорядкой и фактом продаж магазина. Для принятия решения о продлении промо корректнее будет рассматривать фактический запас на магазине.



# Третий вопрос:
На основании данных из таблиц найти:

1)Top-10 SKU в каждом филиале, по которым не хватило товара

2)Top-5 перетаренных магазинов в каждом филиале с указанием наиболее проблемных артикулов

3)Top-10 SKU по иммобилизации средств в запасах на конец промо

Первый пункт - нужно сгруппировать данные по SKU при аггрегировании столбца с посчитанными значениями и вывести результат

In [12]:
# Объединяем две таблицы
data_ful = data.join(RR, on = 'Код СВУ' and "Магазин", lsuffix='_left', rsuffix='_right', how = 'inner')

data_ful.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 250818 entries, 0 to 249054
Data columns (total 58 columns):
 #   Column                                      Non-Null Count   Dtype         
---  ------                                      --------------   -----         
 0   Магазин                                     250818 non-null  int64         
 1   Стадия                                      250818 non-null  object        
 2   Статус                                      250818 non-null  object        
 3   Акция                                       250818 non-null  object        
 4   Начало                                      250818 non-null  datetime64[ns]
 5   Окончание                                   250818 non-null  datetime64[ns]
 6   Вид акции_left                              250818 non-null  object        
 7   Группа_left                                 250818 non-null  object        
 8   Подгруппа_left                              250818 non-null  object       

In [32]:
# Создаём новый столбец с рассчитаным значением для каждой строки

data_ful['overrated'] = data_ful['Факт продаж']- data_ful['План остаток на старт промо (1 расчёт), шт']- data_ful['Разнарядка (1 расчёт) ИТОГ, шт']

# Создаём сводную таблицу с нужной нам группировкой
data_CBY = data_ful.pivot_table(index = ['Филиал_right','Код СВУ_right'],values ='overrated', aggfunc = 'sum')

# Отбрасываем значения, которые нам не нужны
data_CBY.drop(labels = data_CBY[data_CBY['overrated']<=0].index ,axis = 'index',inplace = True)

display(data_CBY)

# Выводим результаты
print('Северо-западный филиал')
display(data_CBY.loc['Северо-Западный'].sort_values(by ='overrated' ,ascending = False).head(10))

print('Центральный филиал')
display(data_CBY.loc['Центральный'].sort_values(by ='overrated' ,ascending = False).head(10))

print('Приволжский филиал')
display(data_CBY.loc['Приволжский'].sort_values(by ='overrated' ,ascending = False).head(10))




overrated
Филиал_right    Код СВУ_right              
Приволжский     103321          2932.162000
                108453          1130.857000
                117893          1557.443000
Северо-Западный 124337          2789.485999
                128582           868.502002
Центральный     4975            6601.419530
                102077          1930.332556
                102083          1678.775687
                103348           178.282978
                103349          7077.308002
                108453           776.958999
                110226          1650.616000
                110753           827.929485
                116573          6617.739001
                117445          1601.191999
                117600          1083.305826
                117704          1149.222998
                117747          5625.682003
                117893          3901.225002
                121160          4707.123998
                121228          2404.435999
                121591          2978.701001
                121712            80.578999
                121773          8671.690997
                124508          1002.799999
                124924          2409.365329
                126140          8815.799000
                128582           848.403000
                128585          1037.793981
                128642           800.124999
                131696          2869.339000
                132583          4743.783160
                133166          8774.933002
                136631          2274.590575
                138845           106.388044
                140396         12146.890056
                142156           740.239001
                142962           763.233002
                145489          1170.727451
                145813           186.293001
                146160          5641.998001
                146282          4578.391000
                148427           411.687000
                150972          1804.309157
                155164          1612.875787
                155166           811.747948
                155168           436.489112
                156058           715.974135
                156080           251.160000
                158581          3853.089271
                158583          2817.572000
                158817           961.155999
                158859          3028.698000

Северо-западный филиал


,overrated
Код СВУ_right,
124337,2789.485999
128582,868.502002


Центральный филиал


,overrated
Код СВУ_right,
140396,12146.890056
126140,8815.799000
133166,8774.933002
121773,8671.690997
103349,7077.308002
116573,6617.739001
4975,6601.419530
146160,5641.998001
117747,5625.682003


Приволжский филиал


,overrated
Код СВУ_right,
103321,2932.162
117893,1557.443
108453,1130.857


In [58]:
# Перейдём ко второму пункту - сгруппируем данные по филиалу и магазину, а потом выведем проблемные товары

data_ful['rem'] = data_ful['Разнарядка (1 расчёт) ИТОГ, шт'] - data_ful['Факт продаж']


data_rem = data_ful[data_ful['Остаток, шт']!=0].pivot_table(index = ['Филиал_right','Магазин_right','Код СВУ_right'],values ='rem', aggfunc = 'sum')

data_rem.drop(labels = data_rem[data_rem['rem']<=0].index ,axis = 'index',inplace = True)

display(data_rem.head(30))

# Выводим результаты
data_CBY.dropna(inplace = True)
print('Центральный филиал')
display(data_rem.loc['Центральный'].sort_values(by ='rem' ,ascending = False).head(5))
print()
print('Центральный филиал подробнее')
print('Магазин 1412')
display(data_rem.loc[('Центральный', 1412)].sort_values(by ='rem' ,ascending = False).head(5))
print('Магазин 1422')
display(data_rem.loc[('Центральный', 1422)].sort_values(by ='rem' ,ascending = False).head(5))
print('Магазин 1449')
display(data_rem.loc[('Центральный', 1449)].sort_values(by ='rem' ,ascending = False).head(5))

print('Приволжский филиал')
display(data_rem.loc['Приволжский'].sort_values(by ='rem' ,ascending = False).head(5))

print('Северо-Западный филиал')
display(data_rem.loc['Северо-Западный'].sort_values(by ='rem' ,ascending = False).head(5))



rem
Филиал_right    Магазин_right Код СВУ_right              
Приволжский     4040          120891           577.954000
                              127726           489.885000
                              128717          3289.571000
Северо-Западный 2199          115955          9333.232000
                              116956          1281.762998
                              119203          4131.153999
                              128585           215.238000
                              141023          9295.988999
Центральный     1412          100213         18798.366000
                              100627          6210.174000
                              100689          5225.123000
                              100690          1359.287001
                              106542          4914.525000
                              107625           515.287000
                              108342           840.756001
                              118578          2129.268000
                              122107          3264.621001
                              131841           945.033000
                              135441         23757.900001
                              135513         10354.084000
                              136587          5154.966999
                              137175          4755.418999
                              137941          7277.600999
                              138247          1082.953000
                              139498          4052.261000
                              144244          2325.996000
                              144528          1525.368000
                              145213          3788.892999
                              145973           796.375001
                              146068          3591.499002

Центральный филиал


rem
Магазин_right Код СВУ_right              
1422          103019         56927.111000
              148184         36538.958000
1412          154673         25975.480000
1449          148184         25759.597000
1412          135441         23757.900001


Центральный филиал подробнее
Магазин 1412


,rem
Код СВУ_right,
154673,25975.480000
135441,23757.900001
100213,18798.366000
156056,17674.307000
135513,10354.084000


...

,rem
Код СВУ_right,
148184,25759.597
144244,9578.890
146621,9178.177
149733,8732.257
146066,5592.734


Приволжский филиал


rem
Магазин_right Код СВУ_right          
4040          128717         3289.571
              120891          577.954
              127726          489.885

Северо-Западный филиал


rem
Магазин_right Код СВУ_right             
2199          115955         9333.232000
              141023         9295.988999
              119203         4131.153999
              116956         1281.762998
              128585          215.238000

In [62]:
# Теперь рассмотрим третьий пункт - вывести топ 10 товаров по иммобилизации средств на конец промо

data_ful['im'] =  data_ful['Цена закупки промо'] *( data_ful['Разнарядка (1 расчёт) ИТОГ, шт'] -data_ful['Факт продаж']) 

data_im = data_ful[data_ful['Прирост, в руб.']<0].pivot_table(index = 'Код СВУ_right',values ='im', aggfunc = 'sum')

data_im.drop(labels = data_im[data_im['im']<=0].index ,axis = 'index',inplace = True)

data_im.sort_values(by = 'im').sort_values(by ='im' ,ascending = False).head(10)

,im
Код СВУ_right,
148184,1.060171e+06
154673,7.408915e+05
135441,7.096698e+05
146621,7.088714e+05
103019,6.709561e+05
146066,4.678616e+05
125052,3.478726e+05
136178,3.466278e+05
141023,3.055609e+05


# Четвёртый вопрос:
Существуют ли такие ситуации, в которых излишний запас, даже с учётом потенциального списания товара, не является основанием для продления промо-акции на товар?

Такие случаи могут быть, например, если акция получила резко негативную критику по каким-либо причинам (например, товар оказался испорченным или некачественным, что привело к отравлению клиентов или чему-то подобному). Другими словами одной из возможных причин может быть недовольство покупателей.

Вторая причина - если акция показала крайне низкие продажи, и размещение на витринах товара промо акции будет идти в убыток, то есть будет выгоднее вообще убрать этот товар и продавать на том месте что-либо другое (например, некоторые продукты со сроком годности до следующего дня захочет покупать довольно мало людей, и было бы более прибыльно разместить на месте этого товара какой-то другой, а также проведение акций с по сути дела просроченными товарами может сильно ударить по репутации магазина, если он находится в условно богатом районе города. 

# Пятый вопрос: 
На основании всех имеющихся данных, а также информации из документа, составьте список пар товар-магазин, по которым   необходимо продлить промо.

Учтём в первую очередь срок годности и остаток товара. Мы знаем, что акция длилась одну неделю. Поэтому примем срок годности для критического уровня в 14 дней на момент начала акции. Не будем учитывать специфику продуктов, таких как выпечка и некоторые молочные продукты. Остаток товара, который надо допрадать в рамках акции и истекающего срока годности, будем смотреть как 20% от начального объёма. 

В данных расчётах сложно увидеть динамику покупок, так как нету данных по дням акции, а также нельзя что-либо сказать о конкретной акции. Единственное что можно учесть, это то, если акция отлично себя показала, перепродажи составили более 50% товара.

Запрос будет иметь вид: Либо товары, где срок годности не более 14 дней и остаток более 15% от начального значения но менее 70%, либо перепродажи более 50%, при этом это промо акция и были сделаны закупки товара. 

In [114]:
data_ful[(((data_ful['Срок годности'] <=14)& 
           ((data_ful['rem']>0) & (data_ful['rem']>0.15*data_ful['Разнарядка (1 расчёт) ИТОГ, шт'])&
            (data_ful['rem']<0.7*data_ful['Разнарядка (1 расчёт) ИТОГ, шт']))) | 
         ((data_ful['Факт продаж'] - 1.5 * data_ful['Разнарядка (1 расчёт) ИТОГ, шт'])
          > 0))&
         (data_ful['Вид акции_right']== "ПРОМО")&(data_ful['Разнарядка (1 расчёт) ИТОГ, шт']!=0)][[
    'Магазин_right','Код СВУ_right']].drop_duplicates()

,Магазин_right,Код СВУ_right
7564,1416,145255
7739,1419,157754
22442,1416,146621
3567,1416,133166
45455,1419,153337
...,...,...
6305,1419,136587
6307,1419,119244
70204,1421,123593
100024,1419,145404


# Шестой вопрос:
В случае продления промо, какой по вашему мнению должен быть оптимальный интервал для продления (единый или индивидуальный)? Как часто в течение этого времени необходимо осуществлять контроль?

В идеальном случае конечно же хотелось бы рассмотреть каждый случай индивидуально, но это физически невозможно для небольшой команды, поэтому лучше будет смотреть по категориям: Если было решено продлить акцию по сроку годности товара и его застоя, то надо выбирать интервал на основе окончания срока годности, если продлеваем промо по причине успешных продаж - то надо ориентироваться на то, сколько шло прошлое промо и сколько товара мы можем продать. Конечно, чтобы не было путаницы со сроками, лучше назначить ряд нормальных значений для проделния - 3 дня, неделя, 2 недели и т.д.

Контроль стоит осуществлять 2 раза, один раз где-то после одной трети акции, второй раз -после двух третьих. Так можно заметить первичную диманику акции и скорректировать что-то, если это необходимо, а также выявить ошибки работников. Второй контроль покажет то, насколько акция имея смысл, и позволит получить данные для последующих решений.